In [1]:
import scipy
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import time

file1 = 'DATA/FLUIDS/CYLINDER_ALL.mat'
file2 = 'DATA/FLUIDS/CYLINDER_basis.mat'

def compute_error(X_true, X_est, order='Fr'):
    diff = abs(X_true - X_est)
    err_a = np.linalg.norm(diff, order)
    err_r = err_a / np.linalg.norm(X_true, order)

    return err_r

def err_loop(X, U, S, Vt, n):
    err_rel = []
    for r in range(n):
        # Construct approximate image
        X_est = U[:,:r] @ S[0:r,:r] @ Vt[:r,:]
        err_rel = compute_error(X, X_est)
        #err_abs.append(err_a)
        err_rel.append(err_r)
    return np.array(err_rel)

In [2]:
'''
(a)
- Compute the SVD of fluid flow data set and plot the singular value spectrum and
the leading singular vector. The U matrix contains eigenflow fields and the S*V.T
represents the amplittudes of these eigenflows as the flow evolves in time
'''

data = scipy.io.loadmat(file1)
#print(data.keys())

X = data['UALL']
print(X.shape)
#print('\n \n ')
#data = scipy.io.loadmat(file2)
#print(data.keys())

U, S_diag, V_t = np.linalg.svd(X, full_matrices = False)
S = np.diag(S_diag)
nn = len(S_diag)


#PART A FIGURES
plt.rcParams['figure.figsize'] = [16, 8]
plt.figure()
plt.semilogy(S_diag, '-o', color='blue')
plt.yticks(np.power(10,np.arange(-16,4,4,dtype=float)))
plt.title('Singular value spectrum')
plt.show()

plt.figure()
plt.plot(S[:,0], '-o', color='green')
plt.title('Leading singular vector')
plt.show()

#Extra to check cumulative energy as rank increases
plt.figure()
plt.plot(np.cumsum(np.diag(S))/np.sum(np.diag(S)), color = 'red')
plt.title('Fraction of Energy captured by Singular modes')
plt.show()




FileNotFoundError: [Errno 2] No such file or directory: 'DATA/FLUIDS/CYLINDER_ALL.mat'

In [ ]:
'''
(b)
- Plot reconstruction movie for various truncation values r.
- Compute the r value needed to capture 90%, 99% and 99.9% of the flow energy based
on the singular value spectrum.  (But energy is -> Frobenious norm squared??)
- Also compute the squared Frobenius norm of the error between the true matrix X,
and the reconstructed matrix X_tilde, where X is the flow field movie
- Plot the movies for each of these truncation values and compare the fidelity
'''

energy_frac = np.cumsum(np.diag(S))/np.sum(np.diag(S))
#print(energy_frac.shape, '  shape of array (fraction using singular values)....')
rankIndex_list = []
a_list = np.array(range(1, len(energy_frac)+1))
#print(a_list.shape)

#RANK WILL BE INDEX + 1
kk=0
for perc in energy_frac:
    if perc >=0.90:
        print('found r to capture 90% @ index', kk)
        rankIndex_list.append(kk)
        break
    kk+=1

for perc in energy_frac[kk:]:
    if perc >=0.99:
        print('found r to capture 99% @ index ', kk)
        rankIndex_list.append(kk)
        break
    kk+=1

for err in energy_frac[kk:]:
    if perc >=0.9999:
        print('found r to capture 99.99% = ', kk)
        rankIndex_list.append(kk)
        break
    kk+=1
    if kk == len(energy_frac):
        print('Could not capture 99.99% based on singular spectrum. Precision error?')

j=0
for r in rankIndex_list:
    #reconstruct X based on rank
    X_r = U[:,:r] @ S[0:r,:r] @ V_t[:r,:]
    fr_err = compute_error(X, X_r, order='Fr')
    print('Frobenious error at @ index ',r, ' is...', fr_err)

    '''
    #comparing movies at each truncation
    plt.figure(j+1)
    img = plt.imshow(X_r)
    #img.set_cmap('gray')
    plt.axis('off')
    plt.title('Rank '+str(r)+' approx.')
    plt.show()
    '''


In [ ]:
'''
(c)
- Fix a value r = 10 and compute the truncated SVD
Each column w_k of the matrix W = S_tild V_tild.T represents
the mixture of the first 10 eigenflows in the kth column of X
- Verify this by comparing the kth snapshot of X with U_tilde w_k
'''
r = 10
U_r = np.copy(U[:,:r])
S_r = np.copy(S[0:r, :r])
Vt_r = np.copy(V_t[0:r,:])
W = S_r @ Vt_r

for k in range(r):
    w_k = np.copy(W[:,k])
    x_k = U_r @ w_k

    X_k = np.copy(X[:,k])
    err = compute_error(X_k, x_k, order=2)
    print('Error at snapshot '+str(k+1) ' is... '+ str(err))


In [ ]:
'''
(d)
- Now build a linear regression model for how the amplitudes w_k evolve in time.
This will be a dynamical system:
w_(k+1) = A w_k

- Create a matrix W with the first 1 through (m-1) columns of S*V.T
and another matrix W' with the 2 through m colums of S*V.T
We will now try to solve for the best-fit A matrix so that
W' = A*W  (approximately)

- Compute the SVD of W and use this to compute the pseudo inverse of W to solve for A
- Compute the eigenvalues of A and plot them in the complex plane
'''